<a href="https://colab.research.google.com/github/Ilvecho/Project_7/blob/main/Project_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import pickle
import re
import random

import torch
from torch import nn
from torch import flatten
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam

from google.colab import files,drive
drive.mount('/content/gdrive')

# get the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/gdrive


In [10]:
data = pd.read_csv('/content/gdrive/MyDrive/P7_files/SalesCRM - CRM.csv')

In [9]:
for col in data.columns:
  print(col)

ID
Country
Education
First Contact
Last Contact
Status
Stage
First Call
Signed up for a demo
Filled in customer survey
Did sign up to the platform
Account Manager assigned
Subscribed


In [11]:
data

,ID,Country,Education,First Contact,Last Contact,Status,Stage,First Call,Signed up for a demo,Filled in customer survey,Did sign up to the platform,Account Manager assigned,Subscribed
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-27,NaN,NaN
1,2,Austria,NaN,NaN,NaN,NaN,NaN,2022-04-28,2022-04-25,2022-04-25,2022-04-25,NaN,NaN
2,3,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-24,NaN,NaN,NaN,NaN
3,4,France,NaN,NaN,NaN,NaN,NaN,2022-04-22,2022-04-20,2022-04-20,2022-04-22,2022-04-22,NaN
4,5,USA,NaN,NaN,NaN,NaN,NaN,2022-04-23,2022-04-19,2022-04-19,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11027,11789,USA,NaN,2022-04-21,2022-04-21,1st message,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11028,11790,USA,NaN,2022-04-21,2022-04-21,1st message,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11029,11791,USA,NaN,2022-04-21,2022-04-21,1st message,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11030,11792,USA,NaN,2022-04-21,2022-04-21,1st message,NaN,NaN,NaN,NaN,NaN,NaN,NaN


from matplotlib import pyplot as plt
data['ID'].plot(kind='hist', bins=20, title='ID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
data.groupby('Status').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
data.groupby('Stage').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['ID']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'ID'}, axis=1)
              .sort_values('ID', ascending=True))
  xs = counted['ID']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = data.sort_values('ID', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Status')):
  _plot_series(series, series_name, i)
  fig.legend(title='Status', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ID')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['ID']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'ID'}, axis=1)
              .sort_values('ID', ascending=True))
  xs = counted['ID']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = data.sort_values('ID', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Stage')):
  _plot_series(series, series_name, i)
  fig.legend(title='Stage', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ID')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
data['ID'].plot(kind='line', figsize=(8, 4), title='ID')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Stage'].value_counts()
    for x_label, grp in data.groupby('Status')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Status')
_ = plt.ylabel('Stage')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(data['Status'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(data, x='ID', y='Status', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(data['Stage'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(data, x='ID', y='Stage', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [16]:
tmp = data.copy()
tmp = tmp.fillna('Missing')

In [25]:
fig = px.bar(
    tmp,
    x=tmp.groupby('Status').size().values,
    y=tmp.groupby('Status').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Status',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

In [24]:
fig = px.bar(
    tmp,
    x=tmp.groupby('Stage').size().values,
    y=tmp.groupby('Stage').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Stage',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()